Nothing too fancy here, just trying to see if I'm clever enoughg to recreate the kaggle example notebook except I'm going to be doing this all in mlx instead of what people would call a "normal" framework. 

Pray for me.

In [1]:
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
import mlx.data as dx

import os
from pprint import pprint

# Input data files are available in this directory.
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./test.csv
./titanic-tutorial.ipynb
./train.csv
./mlxperiment.ipynb
./training key.numbers


In [ ]:
train_data = dx.stream_csv_reader("./train.csv")

first_row = next(train_data)

formatted_row = {key: (type(value), value) for key, value in first_row.items()}

pprint(formatted_row)
# pprint(f"Hello! This passenger's name was: {first_row['Name']}")

# Iterate over the stream and print the first three rows
for i, row in enumerate(train_data):
    if i < 3:
        pprint(row)
    else:
        break

In [ ]:
test_data = dx.stream_csv_reader("./test.csv")

for i, row in enumerate(test_data):
    if i < 3:
        pprint(row)
    else:
        break

Looks to me like it's finally time to create the model!

In [8]:
class DataProcessor:
    def __init__(self, file_path, stats):
        self.file_path = file_path
        self.stats = stats
    
    def read_data(self):
        data_stream = dx.stream_csv_reader(self.file_path)
        return data_stream.sample_transform(self._common_prepare)

    def prepare_for_training(self, batch_size, prefetch_count):
        # Read and preprocess the data
        data_stream = self.read_data()
        # Prepare the data in batches and prefetch for training
        num_threads = prefetch_count if prefetch_count < 4 else 4
        return data_stream.batch(batch_size).prefetch(prefetch_count, num_threads)

    def prepare_for_inference(self):
        # Read and preprocess the data
        data_stream = self.read_data()
        # For inference, directly return the processed data stream
        return data_stream
    
    def _common_prepare(self, sample):
        # Remove ----------------------------------------
        # Remove the 'Name' field
        sample.pop('Name', None)

        # One-hot encode --------------------------------
        embarked = sample.get('Embarked', '')
        embarked_one_hot = {
            'Embarked_S': embarked == 'S',
            'Embarked_C': embarked == 'C',
            'Embarked_Q': embarked == 'Q'
        }
        pclass = sample.get('Pclass', '')
        pclass_one_hot = {
            'Pclass_1': pclass == 1,
            'Pclass_2': pclass == 2,
            'Pclass_3': pclass == 3
        }
        sex = sample.get('Sex', '')
        sex_one_hot = {
            'Sex_male': sex == 'male',
            'Sex_female': sex == 'female'
        }

        # Norrmalize -------------------------------------
        for field in self.stats:
            if field in sample and sample[field] is not None:
                sample[field] = (sample[field] - self.stats[field]['mean']) / self.stats[field]['std']

        # Update the sample with the one-hot encoded fields and return
        return {**sample, **embarked_one_hot, **pclass_one_hot, **sex_one_hot}

Time for a little bit of data analysis to see just how normalized our dataset is!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load in the dataset
df = pd.read_csv('./train.csv')

# Get a statistical summary
print(df.describe())

# Plot the histograms
df.hist()
plt.tight_layout()

In [ ]:
# Show how the test data looks in comparison so we know whether or not the test data is representative of what we got
df_test = pd.read_csv('./test.csv')

# Get a statistical summary
print(df_test.describe())

# Plot the histograms
df_test.hist()
plt.tight_layout()

In [13]:
import pandas as pd
# Load in the dataset
df = pd.read_csv('./train.csv')

# Calculate mean and standard deviation for 'Age', 'SibSp', and 'Parch'
age_mean, age_std = df['Age'].mean(), df['Age'].std()
sibsp_mean, sibsp_std = df['SibSp'].mean(), df['SibSp'].std()
parch_mean, parch_std = df['Parch'].mean(), df['Parch'].std()

# Store the statistics for later use
stats = {
    'Age': {'mean': age_mean, 'std': age_std},
    'SibSp': {'mean': sibsp_mean, 'std': sibsp_std},
    'Parch': {'mean': parch_mean, 'std': parch_mean}
}

# Usage example of DataProcessor
data_processor = DataProcessor("./train.csv", stats)

# For training
train_data = data_processor.prepare_for_training(batch_size=3, prefetch_count=2)

print(train_data)

: 

Now that we have a first revision of the model down, let's work out how we're going to 

In [12]:
print(train_data)

Stream()
